In [1]:
import pandas as pd
import altair as alt

In [2]:
regions = {
    "Africa": "data/Africa_aggregated_data_up_to-2025-10-18.csv",
    "Asia-Pacific": "data/Asia-Pacific_aggregated_data_up_to-2025-10-18.csv",
    "Europe-Central-Asia": "data/Europe-Central-Asia_aggregated_data_up_to-2025-10-18.csv",
    "Latin-America-the-Caribbean": "data/Latin-America-the-Caribbean_aggregated_data_up_to-2025-10-18.csv",
    "Middle-East": "data/Middle-East_aggregated_data_up_to-2025-10-18.csv",
    "US-and-Canada": "data/US-and-Canada_aggregated_data_up_to-2025-10-18.csv"
}

# combine datasets
dfs = {}
for region_name, file_path in regions.items():
    dfs[region_name] = pd.read_csv(file_path)
    print(f"Loaded {region_name}: {len(dfs[region_name])} rows")

# Combine into single dataframe
acled = pd.concat(dfs.values(), ignore_index=True)
print(f"Combined dataset: {len(acled)} total rows")

Loaded Africa: 255889 rows
Loaded Asia-Pacific: 198169 rows
Loaded Europe-Central-Asia: 110481 rows
Loaded Latin-America-the-Caribbean: 161259 rows
Loaded Middle-East: 137579 rows
Loaded US-and-Canada: 20839 rows
Combined dataset: 884216 total rows


## Tier 1 Countries

### Ukraine

In [7]:
# UKRAINE ANALYSIS: ACLED EVENTS vs GOOGLE TRENDS

print("="*80)
print("UKRAINE ANALYSIS")
print("="*80)

# 1. FILTER ACLED DATA
# ---------------------

ukraine_acled = acled[
    (acled['COUNTRY'] == 'Ukraine') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

ukraine_acled['WEEK'] = pd.to_datetime(ukraine_acled['WEEK'])

ukraine_acled['month'] = ukraine_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
ukraine_monthly = ukraine_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

# print(ukraine_acled.head())


print(f"✓ ACLED Data: {len(ukraine_monthly)} months")
print(f"  Date range: {ukraine_monthly['month'].min()} to {ukraine_monthly['month'].max()}")
print(f"  Total events: {ukraine_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {ukraine_monthly['FATALITIES'].sum():,}")


# 2. LOAD GOOGLE TRENDS FILES
# ----------------------------
ukraine_trends_files = {
    'Ukraine conflict': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Ukraine_conflict.csv',
    'Ukraine war': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Ukraine_war.csv',
    'Russia ukraine': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Russia_Ukraine.csv',
    'Ukraine military': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Ukraine_military.csv',
    'Ukraine crisis': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Ukraine_crisis.csv',
    'Ukraine invasion': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Ukraine_invasion.csv'
}

ukraine_trends_data = {}
for name, filepath in ukraine_trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        ukraine_trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"    ✗ Error loading {name}: {e}")
        

# 3. MERGE DATASETS
# -----------------
merged = ukraine_monthly.copy()
for name, df in ukraine_trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )
print(f"\n✓ Merged dataset: {len(merged)} months with {len(ukraine_trends_data)} search terms")


# 4. CORRELATION ANALYSIS
# -----------------------
print("\n" + "="*80)
print("CORRELATION ANALYSIS")
print("="*80)

correlations = []
for term in ukraine_trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

ukraine_corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + ukraine_corr_df.to_string(index=False))
# 5. TIME-LAG ANALYSIS
# --------------------
print("\n" + "="*80)
print("TIME-LAG ANALYSIS")
print("="*80)

ukraine_top_terms = ukraine_corr_df.head(3)['Search Term'].tolist()

for term in ukraine_top_terms:
    print(f"\n{term}:")
    valid_data = merged[['EVENTS', term]].dropna()
    best_corr = -999
    best_lag = 0
    
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

# 6. KEY PERIODS IDENTIFICATION
# -----------------------------
print("\n" + "="*80)
print("KEY PERIODS")
print("="*80)

print("\nTop 5 Event Spikes:")
ukraine_top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(ukraine_trends_data.keys())]
for idx, row in ukraine_top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in ukraine_trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

# 7. VISUALIZATION
# ---------------
print("\n" + "="*80)
print("CREATING VISUALIZATIONS")
print("="*80)

# Normalize data
ukraine_merged_normalized = merged.copy()
ukraine_merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
ukraine_merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
ukraine_plot_data = []
for _, row in ukraine_merged_normalized.iterrows():
    ukraine_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    ukraine_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    for term in ukraine_top_terms:
        if term in row and pd.notna(row[term]):
            ukraine_plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

ukraine_plot_df = pd.DataFrame(ukraine_plot_data)

# Main chart
chart = alt.Chart(ukraine_plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', title='Normalized Value (0-100)', scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', title='Metric', scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N', title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'Ukraine: ACLED Events vs Google Search Interest (2020-2025)',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
).interactive()

chart.save('ukraine_acled_vs_trends.html')
print(f"✓ Saved: ukraine_acled_vs_trends.html")

# Display
chart
# 8. INDIVIDUAL COMPARISON CHARTS
# --------------------------------
for term in ukraine_top_terms:
    ukraine_term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    base = alt.Chart(ukraine_term_data).encode(
        x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', title='ACLED Events', axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    trends_line = base.mark_line(color='red', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='red'), scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'Ukraine: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"ukraine_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

print("\n✓ Ukraine analysis complete!")

UKRAINE ANALYSIS
✓ ACLED Data: 94 months
  Date range: 2017-12-01 00:00:00 to 2025-09-01 00:00:00
  Total events: 90,942
  Total fatalities: 72,434
  ✓ Loaded: Ukraine conflict          - 70 months, max=100.0
  ✓ Loaded: Ukraine war               - 70 months, max=100.0
  ✓ Loaded: Russia ukraine            - 70 months, max=100.0
  ✓ Loaded: Ukraine military          - 70 months, max=100.0
  ✓ Loaded: Ukraine crisis            - 70 months, max=100.0
  ✓ Loaded: Ukraine invasion          - 70 months, max=100.0

✓ Merged dataset: 94 months with 6 search terms

CORRELATION ANALYSIS

     Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
     Ukraine war        0.344672            0.298608           69
Ukraine military        0.152528            0.153151           69
  Russia ukraine        0.004718            0.044032           69
Ukraine invasion       -0.010765            0.031436           69
Ukraine conflict       -0.012067            0.029591           69
  Ukraine crisis  

### India

In [8]:
# INDIA ANALYSIS: ACLED EVENTS vs GOOGLE TRENDS

print("="*80)
print("INDIA ANALYSIS")
print("="*80)

# 1. FILTER ACLED DATA
# ---------------------

india_acled = acled[
    (acled['COUNTRY'] == 'India') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

india_acled['WEEK'] = pd.to_datetime(india_acled['WEEK'])

india_acled['month'] = india_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
india_monthly = india_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

# print(india_acled.head())


print(f"✓ ACLED Data: {len(india_monthly)} months")
print(f"  Date range: {india_monthly['month'].min()} to {india_monthly['month'].max()}")
print(f"  Total events: {india_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {india_monthly['FATALITIES'].sum():,}")


# 2. LOAD GOOGLE TRENDS FILES
# ----------------------------
india_trends_files = {
    'India protests': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_India_protests.csv',
    'India riots': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_India_riots.csv',
    'India demonstrations': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_India_demonstrations.csv',
    'India unrest': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_India_unrest.csv',
    'India strikes': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_India_strikes.csv'
}

india_trends_data = {}
for name, filepath in india_trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        india_trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"    ✗ Error loading {name}: {e}")

# 3. MERGE DATASETS
# -----------------
merged = india_monthly.copy()
for name, df in india_trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )
print(f"\n✓ Merged dataset: {len(merged)} months with {len(india_trends_data)} search terms")

# 4. CORRELATION ANALYSIS
# -----------------------
print("\n" + "="*80)
print("CORRELATION ANALYSIS")
print("="*80)

correlations = []
for term in india_trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

# Then your original line
india_corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)


india_corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + india_corr_df.to_string(index=False))
# 5. TIME-LAG ANALYSIS
# --------------------
print("\n" + "="*80)
print("TIME-LAG ANALYSIS")
print("="*80)

india_top_terms = india_corr_df.head(3)['Search Term'].tolist()

for term in india_top_terms:
    print(f"\n{term}:")
    valid_data = merged[['EVENTS', term]].dropna()
    best_corr = -999
    best_lag = 0
    
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

# 6. KEY PERIODS IDENTIFICATION
# -----------------------------
print("\n" + "="*80)
print("KEY PERIODS")
print("="*80)

print("\nTop 5 Event Spikes:")
india_top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(india_trends_data.keys())]
for idx, row in ukraine_top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in ukraine_trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

# 7. VISUALIZATION
# ---------------
print("\n" + "="*80)
print("CREATING VISUALIZATIONS")
print("="*80)

# Normalize data
india_merged_normalized = merged.copy()
india_merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
india_merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
india_plot_data = []
for _, row in india_merged_normalized.iterrows():
    india_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    india_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    for term in india_top_terms:
        if term in row and pd.notna(row[term]):
            india_plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

india_plot_df = pd.DataFrame(india_plot_data)

# Main chart
chart = alt.Chart(india_plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', title='Normalized Value (0-100)', scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', title='Metric', scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N', title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'India: ACLED Events vs Google Search Interest (2020-2025)',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
).interactive()

chart.save('india_acled_vs_trends.html')
print(f"✓ Saved: india_acled_vs_trends.html")

# Display
chart
# 8. INDIVIDUAL COMPARISON CHARTS
# --------------------------------
for term in india_top_terms:
    india_term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    base = alt.Chart(india_term_data).encode(
        x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', title='ACLED Events', axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    trends_line = base.mark_line(color='red', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='red'), scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'India: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"india_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

print("\n✓ India analysis complete!")

INDIA ANALYSIS
✓ ACLED Data: 118 months
  Date range: 2015-12-01 00:00:00 to 2025-09-01 00:00:00
  Total events: 64,623
  Total fatalities: 4,550
  ✓ Loaded: India protests            - 70 months, max=100
  ✓ Loaded: India riots               - 70 months, max=100
  ✓ Loaded: India demonstrations      - 70 months, max=100
  ✓ Loaded: India unrest              - 70 months, max=100
  ✓ Loaded: India strikes             - 70 months, max=100

✓ Merged dataset: 118 months with 5 search terms

CORRELATION ANALYSIS

         Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
       India strikes        0.040835           -0.063750           69
         India riots       -0.032881            0.274753           69
India demonstrations       -0.052872           -0.127434           69
        India unrest       -0.064392           -0.136305           69
      India protests       -0.093664           -0.118023           69

TIME-LAG ANALYSIS

India strikes:
  Lag -3 months (searches LEAD 

### United States

In [10]:
# UNITED STATES ANALYSIS: ACLED EVENTS vs GOOGLE TRENDS

print("="*80)
print("UNITED STATES ANALYSIS")
print("="*80)

# 1. FILTER ACLED DATA
# ---------------------

us_acled = acled[
    (acled['COUNTRY'] == 'United States') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

us_acled['WEEK'] = pd.to_datetime(us_acled['WEEK'])

us_acled['month'] = us_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
us_monthly = us_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

print(f"✓ ACLED Data: {len(us_monthly)} months")
print(f"  Date range: {us_monthly['month'].min()} to {us_monthly['month'].max()}")
print(f"  Total events: {us_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {us_monthly['FATALITIES'].sum():,}")


# 2. LOAD GOOGLE TRENDS FILES
# ----------------------------
us_trends_files = {
    'US protests': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_US_protests.csv',
    'United states protests': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_United_States_protests.csv',
    'Protests america': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_protests_America.csv',
    'Black lives matter': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Black_Lives_Matter.csv',
    'Capitol riots': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Capitol_riots.csv',
    'January 6': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_January_6.csv'
}

us_trends_data = {}
for name, filepath in us_trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        us_trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"    ✗ Error loading {name}: {e}")
        

# 3. MERGE DATASETS
# -----------------
merged = us_monthly.copy()
for name, df in us_trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )
print(f"\n✓ Merged dataset: {len(merged)} months with {len(us_trends_data)} search terms")


# 4. CORRELATION ANALYSIS
# -----------------------
print("\n" + "="*80)
print("CORRELATION ANALYSIS")
print("="*80)

correlations = []
for term in us_trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

us_corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + us_corr_df.to_string(index=False))
# 5. TIME-LAG ANALYSIS
# --------------------
print("\n" + "="*80)
print("TIME-LAG ANALYSIS")
print("="*80)

us_top_terms = us_corr_df.head(3)['Search Term'].tolist()

for term in us_top_terms:
    print(f"\n{term}:")
    valid_data = merged[['EVENTS', term]].dropna()
    best_corr = -999
    best_lag = 0
    
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

# 6. KEY PERIODS IDENTIFICATION
# -----------------------------
print("\n" + "="*80)
print("KEY PERIODS")
print("="*80)

print("\nTop 5 Event Spikes:")
us_top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(us_trends_data.keys())]
for idx, row in us_top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in us_trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

# 7. VISUALIZATION
# ---------------
print("\n" + "="*80)
print("CREATING VISUALIZATIONS")
print("="*80)

# Normalize data
us_merged_normalized = merged.copy()
us_merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
us_merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
us_plot_data = []
for _, row in us_merged_normalized.iterrows():
    us_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    us_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    for term in us_top_terms:
        if term in row and pd.notna(row[term]):
            us_plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

us_plot_df = pd.DataFrame(us_plot_data)

# Main chart
chart = alt.Chart(us_plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', title='Normalized Value (0-100)', scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', title='Metric', scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N', title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'United States: ACLED Events vs Google Search Interest (2020-2025)',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
).interactive()

chart.save('us_acled_vs_trends.html')
print(f"✓ Saved: us_acled_vs_trends.html")

# Display
chart
# 8. INDIVIDUAL COMPARISON CHARTS
# --------------------------------
for term in us_top_terms:
    us_term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    base = alt.Chart(us_term_data).encode(
        x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', title='ACLED Events', axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    us_line = base.mark_line(color='red', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='red'), scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'United States: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"us_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

print("\n✓ United States analysis complete!")

UNITED STATES ANALYSIS
✓ ACLED Data: 70 months
  Date range: 2019-12-01 00:00:00 to 2025-09-01 00:00:00
  Total events: 30,592
  Total fatalities: 211
  ✓ Loaded: US protests               - 70 months, max=100
  ✓ Loaded: United states protests    - 70 months, max=100
  ✓ Loaded: Protests america          - 70 months, max=100
  ✓ Loaded: Black lives matter        - 70 months, max=100.0
  ✓ Loaded: Capitol riots             - 70 months, max=100.0
  ✓ Loaded: January 6                 - 70 months, max=100

✓ Merged dataset: 70 months with 6 search terms

CORRELATION ANALYSIS

           Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
      Protests america        0.336632            0.103806           69
           US protests        0.226441            0.073965           69
United states protests        0.225036            0.043175           69
    Black lives matter        0.175681            0.080610           69
         Capitol riots       -0.008400           -0.071402 

### Myanmar

In [11]:
# MYANMAR ANALYSIS: ACLED EVENTS vs GOOGLE TRENDS

print("="*80)
print("MYANMAR ANALYSIS")
print("="*80)

# 1. FILTER ACLED DATA
# ---------------------

myanmar_acled = acled[
    (acled['COUNTRY'] == 'Myanmar') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

myanmar_acled['WEEK'] = pd.to_datetime(myanmar_acled['WEEK'])

myanmar_acled['month'] = myanmar_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
myanmar_monthly = myanmar_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()


print(f"✓ ACLED Data: {len(myanmar_monthly)} months")
print(f"  Date range: {myanmar_monthly['month'].min()} to {myanmar_monthly['month'].max()}")
print(f"  Total events: {myanmar_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {myanmar_monthly['FATALITIES'].sum():,}")


# 2. LOAD GOOGLE TRENDS FILES
# ----------------------------
myanmar_trends_files = {
    'Myanmar conflict': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Myanmar_conflict.csv',
    'Myanmar military': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Myanmar_military.csv',
    'Myanmar coup': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Myanmar_coup.csv',
    'Myanmar violence': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Myanmar_violence.csv'
}

myanmar_trends_data = {}
for name, filepath in myanmar_trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        myanmar_trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"    ✗ Error loading {name}: {e}")
        

# 3. MERGE DATASETS
# -----------------
merged = myanmar_monthly.copy()
for name, df in myanmar_trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )
print(f"\n✓ Merged dataset: {len(merged)} months with {len(myanmar_trends_data)} search terms")


# 4. CORRELATION ANALYSIS
# -----------------------
print("\n" + "="*80)
print("CORRELATION ANALYSIS")
print("="*80)

correlations = []
for term in myanmar_trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

myanmar_corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + myanmar_corr_df.to_string(index=False))
# 5. TIME-LAG ANALYSIS
# --------------------
print("\n" + "="*80)
print("TIME-LAG ANALYSIS")
print("="*80)

myanmar_top_terms = myanmar_corr_df.head(3)['Search Term'].tolist()

for term in myanmar_top_terms:
    print(f"\n{term}:")
    valid_data = merged[['EVENTS', term]].dropna()
    best_corr = -999
    best_lag = 0
    
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

# 6. KEY PERIODS IDENTIFICATION
# -----------------------------
print("\n" + "="*80)
print("KEY PERIODS")
print("="*80)

print("\nTop 5 Event Spikes:")
myanmar_top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(myanmar_trends_data.keys())]
for idx, row in myanmar_top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in myanmar_trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

# 7. VISUALIZATION
# ---------------
print("\n" + "="*80)
print("CREATING VISUALIZATIONS")
print("="*80)

# Normalize data
myanmar_merged_normalized = merged.copy()
myanmar_merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
myanmar_merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
myanmar_plot_data = []
for _, row in myanmar_merged_normalized.iterrows():
    myanmar_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    myanmar_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    for term in myanmar_top_terms:
        if term in row and pd.notna(row[term]):
            myanmar_plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

myanmar_plot_df = pd.DataFrame(myanmar_plot_data)

# Main chart
chart = alt.Chart(myanmar_plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', title='Normalized Value (0-100)', scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', title='Metric', scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N', title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'Myanmar: ACLED Events vs Google Search Interest (2020-2025)',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
).interactive()

chart.save('myanmar_acled_vs_trends.html')
print(f"✓ Saved: myanmar_acled_vs_trends.html")

# Display
chart
# 8. INDIVIDUAL COMPARISON CHARTS
# --------------------------------
for term in myanmar_top_terms:
    myanmar_term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    base = alt.Chart(myanmar_term_data).encode(
        x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', title='ACLED Events', axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    trends_line = base.mark_line(color='red', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='red'), scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'Myanmar: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"myanmar_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

print("\n✓ Myanmar analysis complete!")

MYANMAR ANALYSIS
✓ ACLED Data: 187 months
  Date range: 2009-12-01 00:00:00 to 2025-09-01 00:00:00
  Total events: 31,662
  Total fatalities: 32,707
  ✓ Loaded: Myanmar conflict          - 70 months, max=100
  ✓ Loaded: Myanmar military          - 70 months, max=100
  ✓ Loaded: Myanmar coup              - 70 months, max=100.0
  ✓ Loaded: Myanmar violence          - 70 months, max=100

✓ Merged dataset: 187 months with 4 search terms

CORRELATION ANALYSIS

     Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
Myanmar conflict        0.159037            0.165548           69
Myanmar military        0.122164           -0.080312           69
    Myanmar coup        0.069713           -0.123126           69
Myanmar violence        0.058156            0.018507           69

TIME-LAG ANALYSIS

Myanmar conflict:
  Lag -3 months (searches LEAD  ): correlation = +0.344
  Lag -2 months (searches LEAD  ): correlation = +0.295
  Lag -1 months (searches LEAD  ): correlation = +0.268
  La

### Mexico

In [12]:

# MEXICO ANALYSIS: ACLED EVENTS vs GOOGLE TRENDS

print("="*80)
print("MEXICO ANALYSIS")
print("="*80)

# 1. FILTER ACLED DATA
# ---------------------

# us_canada_acled['COUNTRY']

mexico_acled = acled[
    (acled['COUNTRY'] == 'Mexico') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

mexico_acled['WEEK'] = pd.to_datetime(mexico_acled['WEEK'])

mexico_acled['month'] = mexico_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
mexico_monthly = mexico_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()


print(f"✓ ACLED Data: {len(mexico_monthly)} months")
print(f"  Date range: {mexico_monthly['month'].min()} to {mexico_monthly['month'].max()}")
print(f"  Total events: {mexico_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {mexico_monthly['FATALITIES'].sum():,}")


# 2. LOAD GOOGLE TRENDS FILES
# ----------------------------
mexico_trends_files = {
    'Mexico violence': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Mexico_violence.csv',
    'Mexico cartel': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Mexico_cartel.csv',
    'Mexico crime': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Mexico_crime.csv',
    'Mexico drug war': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Mexico_drug_war.csv'
}

mexico_trends_data = {}
for name, filepath in mexico_trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        mexico_trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"    ✗ Error loading {name}: {e}")
        

# 3. MERGE DATASETS
# -----------------
merged = mexico_monthly.copy()
for name, df in mexico_trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )
print(f"\n✓ Merged dataset: {len(merged)} months with {len(mexico_trends_data)} search terms")


# 4. CORRELATION ANALYSIS
# -----------------------
print("\n" + "="*80)
print("CORRELATION ANALYSIS")
print("="*80)

correlations = []
for term in mexico_trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

mexico_corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + mexico_corr_df.to_string(index=False))
# 5. TIME-LAG ANALYSIS
# --------------------
print("\n" + "="*80)
print("TIME-LAG ANALYSIS")
print("="*80)

mexico_top_terms = mexico_corr_df.head(3)['Search Term'].tolist()

for term in mexico_top_terms:
    print(f"\n{term}:")
    valid_data = merged[['EVENTS', term]].dropna()
    best_corr = -999
    best_lag = 0
    
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

# 6. KEY PERIODS IDENTIFICATION
# -----------------------------
print("\n" + "="*80)
print("KEY PERIODS")
print("="*80)

print("\nTop 5 Event Spikes:")
mexico_top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(mexico_trends_data.keys())]
for idx, row in mexico_top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in mexico_trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

# 7. VISUALIZATION
# ---------------
print("\n" + "="*80)
print("CREATING VISUALIZATIONS")
print("="*80)

# Normalize data
mexico_merged_normalized = merged.copy()
mexico_merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
mexico_merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
mexico_plot_data = []
for _, row in mexico_merged_normalized.iterrows():
    mexico_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    mexico_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    for term in mexico_top_terms:
        if term in row and pd.notna(row[term]):
            mexico_plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

mexico_plot_df = pd.DataFrame(mexico_plot_data)

# Main chart
chart = alt.Chart(mexico_plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', title='Normalized Value (0-100)', scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', title='Metric', scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N', title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'Mexico: ACLED Events vs Google Search Interest (2020-2025)',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
).interactive()

chart.save('mexico_acled_vs_trends.html')
print(f"✓ Saved: mexico_acled_vs_trends.html")

# Display
chart
# 8. INDIVIDUAL COMPARISON CHARTS
# --------------------------------
for term in mexico_top_terms:
    mexico_term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    base = alt.Chart(mexico_term_data).encode(
        x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', title='ACLED Events', axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    trends_line = base.mark_line(color='red', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='red'), scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'Mexico: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"mexico_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

print("\n✓ Mexico analysis complete!")

MEXICO ANALYSIS
✓ ACLED Data: 94 months
  Date range: 2017-12-01 00:00:00 to 2025-09-01 00:00:00
  Total events: 37,588
  Total fatalities: 22,233
  ✓ Loaded: Mexico violence           - 70 months, max=100
  ✓ Loaded: Mexico cartel             - 70 months, max=100
  ✓ Loaded: Mexico crime              - 70 months, max=100
  ✓ Loaded: Mexico drug war           - 70 months, max=100

✓ Merged dataset: 94 months with 4 search terms

CORRELATION ANALYSIS

    Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
   Mexico crime        0.120043           -0.050049           69
Mexico violence        0.101397           -0.055221           69
  Mexico cartel        0.077503            0.001002           69
Mexico drug war        0.067364            0.017278           69

TIME-LAG ANALYSIS

Mexico crime:
  Lag -3 months (searches LEAD  ): correlation = +0.059
  Lag -2 months (searches LEAD  ): correlation = +0.026
  Lag -1 months (searches LEAD  ): correlation = +0.174
  Lag +0 months (C

### Syria

In [13]:
# SYRIA ANALYSIS: ACLED EVENTS vs GOOGLE TRENDS

print("="*80)
print("SYRIA ANALYSIS")
print("="*80)

# 1. FILTER ACLED DATA
# ---------------------

syria_acled = acled[
    (acled['COUNTRY'] == 'Syria') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

syria_acled['WEEK'] = pd.to_datetime(syria_acled['WEEK'])

syria_acled['month'] = syria_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
syria_monthly = syria_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

print(f"✓ ACLED Data: {len(syria_monthly)} months")
print(f"  Date range: {syria_monthly['month'].min()} to {syria_monthly['month'].max()}")
print(f"  Total events: {syria_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {syria_monthly['FATALITIES'].sum():,}")


# 2. LOAD GOOGLE TRENDS FILES
# ----------------------------
syria_trends_files = {
    'Syria conflict': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Syria_conflict.csv',
    'Syria war': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Syria_war.csv',
    'Syria military': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Syria_military.csv',
    'Syria bombing': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Syria_bombing.csv'
}

syria_trends_data = {}
for name, filepath in syria_trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        syria_trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"    ✗ Error loading {name}: {e}")
        

# 3. MERGE DATASETS
# -----------------
merged = syria_monthly.copy()
for name, df in syria_trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )
print(f"\n✓ Merged dataset: {len(merged)} months with {len(syria_trends_data)} search terms")


# 4. CORRELATION ANALYSIS
# -----------------------
print("\n" + "="*80)
print("CORRELATION ANALYSIS")
print("="*80)

correlations = []
for term in syria_trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

syria_corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + syria_corr_df.to_string(index=False))
# 5. TIME-LAG ANALYSIS
# --------------------
print("\n" + "="*80)
print("TIME-LAG ANALYSIS")
print("="*80)

syria_top_terms = syria_corr_df.head(3)['Search Term'].tolist()

for term in syria_top_terms:
    print(f"\n{term}:")
    valid_data = merged[['EVENTS', term]].dropna()
    best_corr = -999
    best_lag = 0
    
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

# 6. KEY PERIODS IDENTIFICATION
# -----------------------------
print("\n" + "="*80)
print("KEY PERIODS")
print("="*80)

print("\nTop 5 Event Spikes:")
syria_top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(syria_trends_data.keys())]
for idx, row in syria_top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in syria_trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

# 7. VISUALIZATION
# ---------------
print("\n" + "="*80)
print("CREATING VISUALIZATIONS")
print("="*80)

# Normalize data
syria_merged_normalized = merged.copy()
syria_merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
syria_merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
syria_plot_data = []
for _, row in syria_merged_normalized.iterrows():
    syria_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    syria_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    for term in syria_top_terms:
        if term in row and pd.notna(row[term]):
            syria_plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

syria_plot_df = pd.DataFrame(syria_plot_data)

# Main chart
chart = alt.Chart(syria_plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', title='Normalized Value (0-100)', scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', title='Metric', scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N', title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'Syria: ACLED Events vs Google Search Interest (2020-2025)',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
).interactive()


# Event Annotations

annotation_data = pd.DataFrame({
    'date': [pd.Timestamp('2023-02'), pd.Timestamp('2024-01')],
    'label': ['Earthquake (major death toll)', 'Ongoing civil war dynamics'],
    'text_y': [110, 100],
    'color': ['black','transparent']
})


rules = alt.Chart(annotation_data).mark_rule(
    strokeWidth=2, strokeDash=[5, 5], opacity=1.0
).encode(
    x='date:T',
    color=alt.Color('color:N', scale=None)
)

labels = alt.Chart(annotation_data).mark_text(
    align='center',
    baseline='top',
    dx=10,
    dy=20,
    color='black',
    fontSize=13
).encode(
    x='date:T',
    y='text_y:Q',
    text='label:N'
)

chart = chart + rules + labels


chart.save('syria_acled_vs_trends.html')
print(f"✓ Saved: syria_acled_vs_trends.html")

# Display
chart
# 8. INDIVIDUAL COMPARISON CHARTS
# --------------------------------
for term in syria_top_terms:
    syria_term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    base = alt.Chart(syria_term_data).encode(
        x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', title='ACLED Events', axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    trends_line = base.mark_line(color='red', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='red'), scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'Syria: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"syria_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

print("\n✓ Syria analysis complete!")

SYRIA ANALYSIS
✓ ACLED Data: 106 months
  Date range: 2016-12-01 00:00:00 to 2025-09-01 00:00:00
  Total events: 50,064
  Total fatalities: 46,791
  ✓ Loaded: Syria conflict            - 70 months, max=100
  ✓ Loaded: Syria war                 - 70 months, max=100
  ✓ Loaded: Syria military            - 70 months, max=100
  ✓ Loaded: Syria bombing             - 70 months, max=100

✓ Merged dataset: 106 months with 4 search terms

CORRELATION ANALYSIS

   Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
     Syria war        0.210848            0.375869           69
Syria military        0.076304            0.142612           69
Syria conflict        0.003522            0.123604           69
 Syria bombing       -0.106999           -0.084755           69

TIME-LAG ANALYSIS

Syria war:
  Lag -3 months (searches LEAD  ): correlation = +0.136
  Lag -2 months (searches LEAD  ): correlation = +0.126
  Lag -1 months (searches LEAD  ): correlation = +0.212
  Lag +0 months (CONCURRE

### Brazil

In [14]:
# BRAZIL ANALYSIS: ACLED EVENTS vs GOOGLE TRENDS

print("="*80)
print("BRAZIL ANALYSIS")
print("="*80)

# 1. FILTER ACLED DATA
# ---------------------

brazil_acled = acled[
    (acled['COUNTRY'] == 'Brazil') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

brazil_acled['WEEK'] = pd.to_datetime(brazil_acled['WEEK'])

brazil_acled['month'] = brazil_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
brazil_monthly = brazil_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

print(f"✓ ACLED Data: {len(brazil_monthly)} months")
print(f"  Date range: {brazil_monthly['month'].min()} to {brazil_monthly['month'].max()}")
print(f"  Total events: {brazil_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {brazil_monthly['FATALITIES'].sum():,}")


# 2. LOAD GOOGLE TRENDS FILES
# ----------------------------
brazil_trends_files = {
    'Brazil violence': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Brazil_violence.csv',
    'Brazil gangs': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Brazil_gangs.csv',
    'Brazil conflict': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Brazil_conflict.csv'
}

brazil_trends_data = {}
for name, filepath in brazil_trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        brazil_trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"    ✗ Error loading {name}: {e}")
        

# 3. MERGE DATASETS
# -----------------
merged = brazil_monthly.copy()
for name, df in brazil_trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )
print(f"\n✓ Merged dataset: {len(merged)} months with {len(brazil_trends_data)} search terms")


# 4. CORRELATION ANALYSIS
# -----------------------
print("\n" + "="*80)
print("CORRELATION ANALYSIS")
print("="*80)

correlations = []
for term in brazil_trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

brazil_corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + brazil_corr_df.to_string(index=False))
# 5. TIME-LAG ANALYSIS
# --------------------
print("\n" + "="*80)
print("TIME-LAG ANALYSIS")
print("="*80)

brazil_top_terms = brazil_corr_df.head(3)['Search Term'].tolist()

for term in brazil_top_terms:
    print(f"\n{term}:")
    valid_data = merged[['EVENTS', term]].dropna()
    best_corr = -999
    best_lag = 0
    
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

# 6. KEY PERIODS IDENTIFICATION
# -----------------------------
print("\n" + "="*80)
print("KEY PERIODS")
print("="*80)

print("\nTop 5 Event Spikes:")
brazil_top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(brazil_trends_data.keys())]
for idx, row in brazil_top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in brazil_trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

# 7. VISUALIZATION
# ---------------
print("\n" + "="*80)
print("CREATING VISUALIZATIONS")
print("="*80)

# Normalize data
brazil_merged_normalized = merged.copy()
brazil_merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
brazil_merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
brazil_plot_data = []
for _, row in brazil_merged_normalized.iterrows():
    brazil_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    brazil_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    for term in brazil_top_terms:
        if term in row and pd.notna(row[term]):
            brazil_plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

brazil_plot_df = pd.DataFrame(brazil_plot_data)
# Main chart
chart = alt.Chart(brazil_plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', title='Normalized Value (0-100)', scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', title='Metric', scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N', title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'Brazil: ACLED Events vs Google Search Interest (2020-2025)',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
).interactive()

# Event Annotations

annotation_data = pd.DataFrame({
    'date': [pd.Timestamp('2022-10'), pd.Timestamp('2023-01')],
    'label': ['Presidential election (Lula vs Bolsonaro)', 'Brasília riots'],
    'text_y': [110, 100]
})

rules = alt.Chart(annotation_data).mark_rule(
    strokeWidth=2, strokeDash=[5, 5], color='black', opacity=1.0
).encode(
    x='date:T',
)

labels = alt.Chart(annotation_data).mark_text(
    align='center',
    baseline='top',
    dx=10,
    dy=20,
    color='black',
    fontSize=13
).encode(
    x='date:T',
    y='text_y:Q',
    text='label:N'
)

chart = chart + rules + labels

chart.save('brazil_acled_vs_trends.html')
print(f"✓ Saved: brazil_acled_vs_trends.html")

# Display
chart
# 8. INDIVIDUAL COMPARISON CHARTS
# --------------------------------
for term in brazil_top_terms:
    brazil_term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    base = alt.Chart(brazil_term_data).encode(
        x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', title='ACLED Events', axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    trends_line = base.mark_line(color='red', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='red'), scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'Brazil: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"brazil_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

print("\n✓ Brail analysis complete!")

BRAZIL ANALYSIS
✓ ACLED Data: 94 months
  Date range: 2017-12-01 00:00:00 to 2025-09-01 00:00:00
  Total events: 35,287
  Total fatalities: 18,930
  ✓ Loaded: Brazil violence           - 70 months, max=100
  ✓ Loaded: Brazil gangs              - 70 months, max=100
  ✓ Loaded: Brazil conflict           - 70 months, max=100

✓ Merged dataset: 94 months with 3 search terms

CORRELATION ANALYSIS

    Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
   Brazil gangs        0.027155            0.075197           69
Brazil violence       -0.022394            0.014697           69
Brazil conflict       -0.026456            0.042847           69

TIME-LAG ANALYSIS

Brazil gangs:
  Lag -3 months (searches LEAD  ): correlation = -0.001
  Lag -2 months (searches LEAD  ): correlation = +0.045
  Lag -1 months (searches LEAD  ): correlation = +0.050
  Lag +0 months (CONCURRENT     ): correlation = +0.027
  Lag +1 months (searches LAG   ): correlation = +0.050
  Lag +2 months (searches LAG 

### Palestine

In [15]:
# PALESTINE ANALYSIS: ACLED EVENTS vs GOOGLE TRENDS

print("="*80)
print("PALESTINE ANALYSIS")
print("="*80)

# 1. FILTER ACLED DATA
# ---------------------

palestine_acled = acled[
    (acled['COUNTRY'] == 'Palestine') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

palestine_acled['WEEK'] = pd.to_datetime(palestine_acled['WEEK'])

palestine_acled['month'] = palestine_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
palestine_monthly = palestine_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

print(f"✓ ACLED Data: {len(palestine_monthly)} months")
print(f"  Date range: {palestine_monthly['month'].min()} to {palestine_monthly['month'].max()}")
print(f"  Total events: {palestine_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {palestine_monthly['FATALITIES'].sum():,}")


# 2. LOAD GOOGLE TRENDS FILES
# ----------------------------
palestine_trends_files = {
    'Palestine conflict': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Palestine_conflict.csv',
    'Israel palestine': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Israel_Palestine.csv',
    'Gaza conflict': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Gaza_conflict.csv',
    'Gaza ceasefire': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Gaza_ceasefire.csv'
}

palestine_trends_data = {}
for name, filepath in palestine_trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        palestine_trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"    ✗ Error loading {name}: {e}")
        

# 3. MERGE DATASETS
# -----------------
merged = palestine_monthly.copy()
for name, df in palestine_trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )
print(f"\n✓ Merged dataset: {len(merged)} months with {len(palestine_trends_data)} search terms")


# 4. CORRELATION ANALYSIS
# -----------------------
print("\n" + "="*80)
print("CORRELATION ANALYSIS")
print("="*80)

correlations = []
for term in palestine_trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

palestine_corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + palestine_corr_df.to_string(index=False))
# 5. TIME-LAG ANALYSIS
# --------------------
print("\n" + "="*80)
print("TIME-LAG ANALYSIS")
print("="*80)

palestine_top_terms = palestine_corr_df.head(3)['Search Term'].tolist()

for term in palestine_top_terms:
    print(f"\n{term}:")
    valid_data = merged[['EVENTS', term]].dropna()
    best_corr = -999
    best_lag = 0
    
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

# 6. KEY PERIODS IDENTIFICATION
# -----------------------------
print("\n" + "="*80)
print("KEY PERIODS")
print("="*80)

print("\nTop 5 Event Spikes:")
palestine_top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(palestine_trends_data.keys())]
for idx, row in palestine_top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in palestine_trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

# 7. VISUALIZATION
# ---------------
print("\n" + "="*80)
print("CREATING VISUALIZATIONS")
print("="*80)

# Normalize data
palestine_merged_normalized = merged.copy()
palestine_merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
palestine_merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
palestine_plot_data = []
for _, row in palestine_merged_normalized.iterrows():
    palestine_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    palestine_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    for term in palestine_top_terms:
        if term in row and pd.notna(row[term]):
            palestine_plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

palestine_plot_df = pd.DataFrame(palestine_plot_data)

# Main chart
chart = alt.Chart(palestine_plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', title='Normalized Value (0-100)', scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', title='Metric', scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N', title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'Palestine: ACLED Events vs Google Search Interest (2020-2025)',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
).interactive()

chart.save('palestine_acled_vs_trends.html')
print(f"✓ Saved: palestine_acled_vs_trends.html")

# Display
chart
# 8. INDIVIDUAL COMPARISON CHARTS
# --------------------------------
for term in palestine_top_terms:
    palestine_term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    base = alt.Chart(palestine_term_data).encode(
        x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', title='ACLED Events', axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    trends_line = base.mark_line(color='red', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='red'), scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'Palestine: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"palestine_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

print("\n✓ Palestine analysis complete!")

PALESTINE ANALYSIS
✓ ACLED Data: 118 months
  Date range: 2015-12-01 00:00:00 to 2025-09-01 00:00:00
  Total events: 27,743
  Total fatalities: 22,882
  ✓ Loaded: Palestine conflict        - 70 months, max=100
  ✓ Loaded: Israel palestine          - 70 months, max=100
  ✓ Loaded: Gaza conflict             - 70 months, max=100.0
  ✓ Loaded: Gaza ceasefire            - 70 months, max=100.0

✓ Merged dataset: 118 months with 4 search terms

CORRELATION ANALYSIS

       Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
     Gaza conflict        0.595142            0.747575           69
  Israel palestine        0.554558            0.790732           69
Palestine conflict        0.485693            0.718237           69
    Gaza ceasefire        0.367205            0.288803           69

TIME-LAG ANALYSIS

Gaza conflict:
  Lag -3 months (searches LEAD  ): correlation = +0.603
  Lag -2 months (searches LEAD  ): correlation = +0.603
  Lag -1 months (searches LEAD  ): correlation = 

### Yemen

In [16]:
# YEMEN ANALYSIS: ACLED EVENTS vs GOOGLE TRENDS

print("="*80)
print("YEMEN ANALYSIS")
print("="*80)

# 1. FILTER ACLED DATA
# ---------------------

yemen_acled = acled[
    (acled['COUNTRY'] == 'Yemen') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

yemen_acled['WEEK'] = pd.to_datetime(yemen_acled['WEEK'])

yemen_acled['month'] = yemen_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
yemen_monthly = yemen_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

print(f"✓ ACLED Data: {len(yemen_monthly)} months")
print(f"  Date range: {yemen_monthly['month'].min()} to {yemen_monthly['month'].max()}")
print(f"  Total events: {yemen_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {yemen_monthly['FATALITIES'].sum():,}")


# 2. LOAD GOOGLE TRENDS FILES
# ----------------------------
yemen_trends_files = {
    'Yemen conflict': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Yemen_conflict.csv',
    'Yemen war': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Yemen_war.csv',
    'Yemen violence': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Yemen_violence.csv',
    'Houthi yemen': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Houthi_Yemen.csv'
}

yemen_trends_data = {}
for name, filepath in yemen_trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        yemen_trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"    ✗ Error loading {name}: {e}")
        

# 3. MERGE DATASETS
# -----------------
merged = yemen_monthly.copy()
for name, df in yemen_trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )
print(f"\n✓ Merged dataset: {len(merged)} months with {len(yemen_trends_data)} search terms")


# 4. CORRELATION ANALYSIS
# -----------------------
print("\n" + "="*80)
print("CORRELATION ANALYSIS")
print("="*80)

correlations = []
for term in yemen_trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

yemen_corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + yemen_corr_df.to_string(index=False))
# 5. TIME-LAG ANALYSIS
# --------------------
print("\n" + "="*80)
print("TIME-LAG ANALYSIS")
print("="*80)

yemen_top_terms = yemen_corr_df.head(3)['Search Term'].tolist()

for term in yemen_top_terms:
    print(f"\n{term}:")
    valid_data = merged[['EVENTS', term]].dropna()
    best_corr = -999
    best_lag = 0
    
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

# 6. KEY PERIODS IDENTIFICATION
# -----------------------------
print("\n" + "="*80)
print("KEY PERIODS")
print("="*80)

print("\nTop 5 Event Spikes:")
yemen_top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(yemen_trends_data.keys())]
for idx, row in yemen_top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in yemen_trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

# 7. VISUALIZATION
# ---------------
print("\n" + "="*80)
print("CREATING VISUALIZATIONS")
print("="*80)

# Normalize data
yemen_merged_normalized = merged.copy()
yemen_merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
yemen_merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
yemen_plot_data = []
for _, row in yemen_merged_normalized.iterrows():
    yemen_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    yemen_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    for term in yemen_top_terms:
        if term in row and pd.notna(row[term]):
            yemen_plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

yemen_plot_df = pd.DataFrame(yemen_plot_data)

# Main chart
chart = alt.Chart(yemen_plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', title='Normalized Value (0-100)', scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', title='Metric', scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N', title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'Yemen: ACLED Events vs Google Search Interest (2020-2025)',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
).interactive()


# Event Annotations

annotation_data = pd.DataFrame({
    'date': [pd.Timestamp('2024-01'), pd.Timestamp('2024-05')],
    'label': ['Red Sea attacks (Houthu targeting ships)', 'Ongoing civil war'],
    'text_y': [110, 115],
    'color': ['black', 'transparent']
})

rules = alt.Chart(annotation_data).mark_rule(
    strokeWidth=2, strokeDash=[5, 5], opacity=1.0
).encode(
    x='date:T',
    color=alt.Color('color:N', scale=None)
)

labels = alt.Chart(annotation_data).mark_text(
    align='center',
    baseline='top',
    dx=10,
    dy=20,
    color='black',
    fontSize=13
).encode(
    x='date:T',
    y='text_y:Q',
    text='label:N'
)

chart = chart + rules + labels

chart.save('yemen_acled_vs_trends.html')
print(f"✓ Saved: yemen_acled_vs_trends.html")

# Display
chart
# 8. INDIVIDUAL COMPARISON CHARTS
# --------------------------------
for term in yemen_top_terms:
    yemen_term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    base = alt.Chart(yemen_term_data).encode(
        x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', title='ACLED Events', axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    trends_line = base.mark_line(color='red', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='red'), scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'Yemen: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"yemen_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

print("\n✓ Yemen analysis complete!")

YEMEN ANALYSIS
✓ ACLED Data: 130 months
  Date range: 2014-12-01 00:00:00 to 2025-09-01 00:00:00
  Total events: 36,708
  Total fatalities: 55,897
  ✓ Loaded: Yemen conflict            - 70 months, max=100
  ✓ Loaded: Yemen war                 - 70 months, max=100
  ✓ Loaded: Yemen violence            - 70 months, max=100
  ✓ Loaded: Houthi yemen              - 70 months, max=100

✓ Merged dataset: 130 months with 4 search terms

CORRELATION ANALYSIS

   Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
  Houthi yemen        0.078354           -0.240244           69
Yemen violence        0.068332           -0.256933           69
Yemen conflict        0.046719           -0.247660           69
     Yemen war       -0.082668           -0.195090           69

TIME-LAG ANALYSIS

Houthi yemen:
  Lag -3 months (searches LEAD  ): correlation = +0.085
  Lag -2 months (searches LEAD  ): correlation = +0.076
  Lag -1 months (searches LEAD  ): correlation = +0.076
  Lag +0 months (CONCU

### Pakistan

In [17]:
# PAKISTAN ANALYSIS: ACLED EVENTS vs GOOGLE TRENDS

print("="*80)
print("PAKISTAN ANALYSIS")
print("="*80)

# 1. FILTER ACLED DATA
# ---------------------

pakistan_acled = acled[
    (acled['COUNTRY'] == 'Pakistan') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

pakistan_acled['WEEK'] = pd.to_datetime(pakistan_acled['WEEK'])

pakistan_acled['month'] = pakistan_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
pakistan_monthly = pakistan_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

print(f"✓ ACLED Data: {len(pakistan_monthly)} months")
print(f"  Date range: {pakistan_monthly['month'].min()} to {pakistan_monthly['month'].max()}")
print(f"  Total events: {pakistan_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {pakistan_monthly['FATALITIES'].sum():,}")


# 2. LOAD GOOGLE TRENDS FILES
# ----------------------------
pakistan_trends_files = {
    'Pakistan protests': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Pakistan_protests.csv',
    'Pakistan violence': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Pakistan_violence.csv',
    'Pakistan demonstrations': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Pakistan_demonstrations.csv',
    'Pakistan unrest': 'data/google_trends_raw/TIER1_COUNTRIES/google_trends_Pakistan_unrest.csv'
}

pakistan_trends_data = {}
for name, filepath in pakistan_trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        pakistan_trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"    ✗ Error loading {name}: {e}")
        

# 3. MERGE DATASETS
# -----------------
merged = pakistan_monthly.copy()
for name, df in pakistan_trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )
print(f"\n✓ Merged dataset: {len(merged)} months with {len(pakistan_trends_data)} search terms")


# 4. CORRELATION ANALYSIS
# -----------------------
print("\n" + "="*80)
print("CORRELATION ANALYSIS")
print("="*80)

correlations = []
for term in pakistan_trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

pakistan_corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + pakistan_corr_df.to_string(index=False))
# 5. TIME-LAG ANALYSIS
# --------------------
print("\n" + "="*80)
print("TIME-LAG ANALYSIS")
print("="*80)

pakistan_top_terms = pakistan_corr_df.head(3)['Search Term'].tolist()

for term in pakistan_top_terms:
    print(f"\n{term}:")
    valid_data = merged[['EVENTS', term]].dropna()
    best_corr = -999
    best_lag = 0
    
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

# 6. KEY PERIODS IDENTIFICATION
# -----------------------------
print("\n" + "="*80)
print("KEY PERIODS")
print("="*80)

print("\nTop 5 Event Spikes:")
pakistan_top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(pakistan_trends_data.keys())]
for idx, row in pakistan_top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in pakistan_trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

# 7. VISUALIZATION
# ---------------
print("\n" + "="*80)
print("CREATING VISUALIZATIONS")
print("="*80)

# Normalize data
pakistan_merged_normalized = merged.copy()
pakistan_merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
pakistan_merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
pakistan_plot_data = []
for _, row in pakistan_merged_normalized.iterrows():
    pakistan_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    pakistan_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    for term in pakistan_top_terms:
        if term in row and pd.notna(row[term]):
            pakistan_plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

pakistan_plot_df = pd.DataFrame(pakistan_plot_data)

# Main chart
chart = alt.Chart(pakistan_plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', title='Normalized Value (0-100)', scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', title='Metric', scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N', title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'Pakistan: ACLED Events vs Google Search Interest (2020-2025)',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
).interactive()


# Event Annotations

annotation_data = pd.DataFrame({
    'date': [pd.Timestamp('2023-10-07'), pd.Timestamp('2023-10')],
    'label': ['Hamas attacks', 'Gaza war escalation'],
    'text_y': [110, 100]
})

rules = alt.Chart(annotation_data).mark_rule(
    strokeWidth=2, strokeDash=[5, 5], color='black', opacity=1.0
).encode(
    x='date:T',
)

labels = alt.Chart(annotation_data).mark_text(
    align='center',
    baseline='top',
    dx=10,
    dy=20,
    color='black',
    fontSize=13
).encode(
    x='date:T',
    y='text_y:Q',
    text='label:N'
)

chart = chart + rules + labels


chart.save('pakistan_acled_vs_trends.html')
print(f"✓ Saved: pakistan_acled_vs_trends.html")

# Display
chart
# 8. INDIVIDUAL COMPARISON CHARTS
# --------------------------------
for term in pakistan_top_terms:
    pakistan_term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    base = alt.Chart(pakistan_term_data).encode(
        x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', title='ACLED Events', axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    trends_line = base.mark_line(color='red', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='red'), scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'Pakistan: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"pakistan_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

print("\n✓ Pakistan analysis complete!")

PAKISTAN ANALYSIS
✓ ACLED Data: 190 months
  Date range: 2009-12-01 00:00:00 to 2025-09-01 00:00:00
  Total events: 37,437
  Total fatalities: 18,582
  ✓ Loaded: Pakistan protests         - 70 months, max=100
  ✓ Loaded: Pakistan violence         - 70 months, max=100
  ✓ Loaded: Pakistan demonstrations   - 70 months, max=100
  ✓ Loaded: Pakistan unrest           - 70 months, max=100

✓ Merged dataset: 190 months with 4 search terms

CORRELATION ANALYSIS

            Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
        Pakistan unrest        0.094994            0.349496           69
Pakistan demonstrations        0.090155            0.307994           69
      Pakistan protests        0.087071            0.300604           69
      Pakistan violence        0.048332            0.260964           69

TIME-LAG ANALYSIS

Pakistan unrest:
  Lag -3 months (searches LEAD  ): correlation = +0.052
  Lag -2 months (searches LEAD  ): correlation = +0.061
  Lag -1 months (searches L

## Tier 2 Countries

### Iraq

In [19]:
# IRAQ ANALYSIS: ACLED EVENTS vs GOOGLE TRENDS

print("="*80)
print("IRAQ ANALYSIS")
print("="*80)

# 1. FILTER ACLED DATA
# ---------------------

iraq_acled = acled[
    (acled['COUNTRY'] == 'Iraq') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

iraq_acled['WEEK'] = pd.to_datetime(iraq_acled['WEEK'])

iraq_acled['month'] = iraq_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
iraq_monthly = iraq_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()


print(f"✓ ACLED Data: {len(iraq_monthly)} months")
print(f"  Date range: {iraq_monthly['month'].min()} to {iraq_monthly['month'].max()}")
print(f"  Total events: {iraq_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {iraq_monthly['FATALITIES'].sum():,}")


# 2. LOAD GOOGLE TRENDS FILES
# ----------------------------
iraq_trends_files = {
    'Iraq conflict': 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_Iraq_conflict.csv',
    'Iraq war': 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_Iraq_war.csv',
    'Iraq military': 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_Iraq_military.csv'
}

iraq_trends_data = {}
for name, filepath in iraq_trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        iraq_trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"    ✗ Error loading {name}: {e}")
        

# 3. MERGE DATASETS
# -----------------
merged = iraq_monthly.copy()
for name, df in iraq_trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )
print(f"\n✓ Merged dataset: {len(merged)} months with {len(iraq_trends_data)} search terms")


# 4. CORRELATION ANALYSIS
# -----------------------
print("\n" + "="*80)
print("CORRELATION ANALYSIS")
print("="*80)

correlations = []
for term in iraq_trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

iraq_corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + iraq_corr_df.to_string(index=False))


# 5. TIME-LAG ANALYSIS
# --------------------
print("\n" + "="*80)
print("TIME-LAG ANALYSIS")
print("="*80)

iraq_top_terms = iraq_corr_df.head(3)['Search Term'].tolist()

for term in iraq_top_terms:
    print(f"\n{term}:")
    valid_data = merged[['EVENTS', term]].dropna()
    best_corr = -999
    best_lag = 0
    
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

# 6. KEY PERIODS IDENTIFICATION
# -----------------------------
print("\n" + "="*80)
print("KEY PERIODS")
print("="*80)

print("\nTop 5 Event Spikes:")
iraq_top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(iraq_trends_data.keys())]
for idx, row in iraq_top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in iraq_trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

# 7. VISUALIZATION
# ---------------
print("\n" + "="*80)
print("CREATING VISUALIZATIONS")
print("="*80)

# Normalize data
iraq_merged_normalized = merged.copy()
iraq_merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
iraq_merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
iraq_plot_data = []
for _, row in iraq_merged_normalized.iterrows():
    iraq_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    iraq_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    for term in iraq_top_terms:
        if term in row and pd.notna(row[term]):
            iraq_plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

iraq_plot_df = pd.DataFrame(iraq_plot_data)

# Main chart
chart = alt.Chart(iraq_plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', title='Normalized Value (0-100)', scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', title='Metric', scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N', title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'Iraq: ACLED Events vs Google Search Interest (2020-2025)',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
).interactive()

chart.save('iraq_acled_vs_trends.html')
print(f"✓ Saved: iraq_acled_vs_trends.html")

# Display
chart

# 8. INDIVIDUAL COMPARISON CHARTS
# --------------------------------
for term in iraq_top_terms:
    iraq_term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    base = alt.Chart(iraq_term_data).encode(
        x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', title='ACLED Events', axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    trends_line = base.mark_line(color='red', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='red'), scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'Iraq: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"Iraq_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

print("\n✓ Iraq analysis complete!")

IRAQ ANALYSIS
✓ ACLED Data: 118 months
  Date range: 2015-12-01 00:00:00 to 2025-09-01 00:00:00
  Total events: 22,556
  Total fatalities: 36,368
  ✓ Loaded: Iraq conflict             - 70 months, max=100
  ✓ Loaded: Iraq war                  - 70 months, max=100
  ✓ Loaded: Iraq military             - 70 months, max=100

✓ Merged dataset: 118 months with 3 search terms

CORRELATION ANALYSIS

  Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
     Iraq war       -0.134293           -0.244893           69
Iraq conflict       -0.151399           -0.358261           69
Iraq military       -0.166390           -0.267034           69

TIME-LAG ANALYSIS

Iraq war:
  Lag -3 months (searches LEAD  ): correlation = -0.039
  Lag -2 months (searches LEAD  ): correlation = -0.036
  Lag -1 months (searches LEAD  ): correlation = -0.044
  Lag +0 months (CONCURRENT     ): correlation = -0.134
  Lag +1 months (searches LAG   ): correlation = -0.044
  Lag +2 months (searches LAG   ): correla

### France

In [20]:
# FRANCE ANALYSIS: ACLED EVENTS vs GOOGLE TRENDS

print("="*80)
print("FRANCE ANALYSIS")
print("="*80)

# 1. FILTER ACLED DATA
# ---------------------

france_acled = acled[
    (acled['COUNTRY'] == 'France') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

france_acled['WEEK'] = pd.to_datetime(france_acled['WEEK'])

france_acled['month'] = france_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
france_monthly = france_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

# print(ukraine_acled.head())


print(f"✓ ACLED Data: {len(france_monthly)} months")
print(f"  Date range: {france_monthly['month'].min()} to {france_monthly['month'].max()}")
print(f"  Total events: {france_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {france_monthly['FATALITIES'].sum():,}")


# 2. LOAD GOOGLE TRENDS FILES
# ----------------------------
france_trends_files = {
    'france protests': 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_France_protests.csv',
    'france demonstrations': 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_France_demonstrations.csv'
}

france_trends_data = {}
for name, filepath in france_trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        france_trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"    ✗ Error loading {name}: {e}")
        

# 3. MERGE DATASETS
# -----------------
merged = france_monthly.copy()
for name, df in france_trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )
print(f"\n✓ Merged dataset: {len(merged)} months with {len(france_trends_data)} search terms")


# 4. CORRELATION ANALYSIS
# -----------------------
print("\n" + "="*80)
print("CORRELATION ANALYSIS")
print("="*80)

correlations = []
for term in france_trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

france_corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + france_corr_df.to_string(index=False))


# 5. TIME-LAG ANALYSIS
# --------------------
print("\n" + "="*80)
print("TIME-LAG ANALYSIS")
print("="*80)

france_top_terms = france_corr_df.head(3)['Search Term'].tolist()

for term in france_top_terms:
    print(f"\n{term}:")
    valid_data = merged[['EVENTS', term]].dropna()
    best_corr = -999
    best_lag = 0
    
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

# 6. KEY PERIODS IDENTIFICATION
# -----------------------------
print("\n" + "="*80)
print("KEY PERIODS")
print("="*80)

print("\nTop 5 Event Spikes:")
france_top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(france_trends_data.keys())]
for idx, row in france_top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in france_trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

# 7. VISUALIZATION
# ---------------
print("\n" + "="*80)
print("CREATING VISUALIZATIONS")
print("="*80)

# Normalize data
france_merged_normalized = merged.copy()
france_merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
france_merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
france_plot_data = []
for _, row in france_merged_normalized.iterrows():
    france_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    france_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    for term in france_top_terms:
        if term in row and pd.notna(row[term]):
            france_plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

france_plot_df = pd.DataFrame(france_plot_data)

# Main chart
chart = alt.Chart(france_plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', title='Normalized Value (0-100)', scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', title='Metric', scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N', title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'France: ACLED Events vs Google Search Interest (2020-2025)',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
).interactive()

chart.save('france_acled_vs_trends.html')
print(f"✓ Saved: france_acled_vs_trends.html")

# Display
chart

# 8. INDIVIDUAL COMPARISON CHARTS
# --------------------------------
for term in france_top_terms:
    france_term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    base = alt.Chart(france_term_data).encode(
        x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', title='ACLED Events', axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    trends_line = base.mark_line(color='red', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='red'), scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'France: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"france_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

print("\n✓ France analysis complete!")

FRANCE ANALYSIS
✓ ACLED Data: 70 months
  Date range: 2019-12-01 00:00:00 to 2025-09-01 00:00:00
  Total events: 13,965
  Total fatalities: 20
  ✓ Loaded: france protests           - 70 months, max=100
    ✗ Error loading france demonstrations: [Errno 2] No such file or directory: 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_France_demonstrations.csv'

✓ Merged dataset: 70 months with 1 search terms

CORRELATION ANALYSIS

    Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
france protests        0.222389           -0.097553           69

TIME-LAG ANALYSIS

france protests:
  Lag -3 months (searches LEAD  ): correlation = +0.255
  Lag -2 months (searches LEAD  ): correlation = +0.257
  Lag -1 months (searches LEAD  ): correlation = +0.234
  Lag +0 months (CONCURRENT     ): correlation = +0.222
  Lag +1 months (searches LAG   ): correlation = +0.234
  Lag +2 months (searches LAG   ): correlation = +0.257
  Lag +3 months (searches LAG   ): correlation = +0.255

  → B

### Russia

In [21]:
# RUSSIA ANALYSIS: ACLED EVENTS vs GOOGLE TRENDS

print("="*80)
print("RUSSIA ANALYSIS")
print("="*80)

# 1. FILTER ACLED DATA
# ---------------------

russia_acled = acled[
    (acled['COUNTRY'] == 'Russia') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

russia_acled['WEEK'] = pd.to_datetime(russia_acled['WEEK'])

russia_acled['month'] = russia_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
russia_monthly = russia_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

print(f"✓ ACLED Data: {len(russia_monthly)} months")
print(f"  Date range: {russia_monthly['month'].min()} to {russia_monthly['month'].max()}")
print(f"  Total events: {russia_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {russia_monthly['FATALITIES'].sum():,}")


# 2. LOAD GOOGLE TRENDS FILES
# ----------------------------
russia_trends_files = {
    'Russia military': 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_Russia_military.csv',
    'Russia conflict': 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_Russia_conflict.csv'
}


russia_trends_data = {}
for name, filepath in russia_trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        russia_trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"    ✗ Error loading {name}: {e}")
        

# 3. MERGE DATASETS
# -----------------
merged = russia_monthly.copy()
for name, df in russia_trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )
print(f"\n✓ Merged dataset: {len(merged)} months with {len(russia_trends_data)} search terms")


# 4. CORRELATION ANALYSIS
# -----------------------
print("\n" + "="*80)
print("CORRELATION ANALYSIS")
print("="*80)

correlations = []
for term in russia_trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

russia_corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + russia_corr_df.to_string(index=False))


# 5. TIME-LAG ANALYSIS
# --------------------
print("\n" + "="*80)
print("TIME-LAG ANALYSIS")
print("="*80)

russia_top_terms = russia_corr_df.head(3)['Search Term'].tolist()

for term in russia_top_terms:
    print(f"\n{term}:")
    valid_data = merged[['EVENTS', term]].dropna()
    best_corr = -999
    best_lag = 0
    
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

# 6. KEY PERIODS IDENTIFICATION
# -----------------------------
print("\n" + "="*80)
print("KEY PERIODS")
print("="*80)

print("\nTop 5 Event Spikes:")
russia_top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(russia_trends_data.keys())]
for idx, row in russia_top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in russia_trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

# 7. VISUALIZATION
# ---------------
print("\n" + "="*80)
print("CREATING VISUALIZATIONS")
print("="*80)

# Normalize data
russia_merged_normalized = merged.copy()
russia_merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
russia_merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
russia_plot_data = []
for _, row in russia_merged_normalized.iterrows():
    russia_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    russia_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    for term in russia_top_terms:
        if term in row and pd.notna(row[term]):
            russia_plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

russia_plot_df = pd.DataFrame(russia_plot_data)

# Main chart
chart = alt.Chart(russia_plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', title='Normalized Value (0-100)', scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', title='Metric', scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N', title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'Russia: ACLED Events vs Google Search Interest (2020-2025)',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
).interactive()

chart.save('russia_acled_vs_trends.html')
print(f"✓ Saved: russia_acled_vs_trends.html")

# Display
chart

# 8. INDIVIDUAL COMPARISON CHARTS
# --------------------------------
for term in russia_top_terms:
    russia_term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    base = alt.Chart(russia_term_data).encode(
        x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', title='ACLED Events', axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    trends_line = base.mark_line(color='red', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='red'), scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'Russia: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"russia_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

print("\n✓ Russia analysis complete!")

RUSSIA ANALYSIS
✓ ACLED Data: 94 months
  Date range: 2017-12-01 00:00:00 to 2025-09-01 00:00:00
  Total events: 15,656
  Total fatalities: 2,060
  ✓ Loaded: Russia military           - 70 months, max=100
  ✓ Loaded: Russia conflict           - 70 months, max=100.0

✓ Merged dataset: 94 months with 2 search terms

CORRELATION ANALYSIS

    Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
Russia military        0.030608           -0.048462           69
Russia conflict       -0.072322           -0.076564           69

TIME-LAG ANALYSIS

Russia military:
  Lag -3 months (searches LEAD  ): correlation = -0.044
  Lag -2 months (searches LEAD  ): correlation = -0.041
  Lag -1 months (searches LEAD  ): correlation = +0.010
  Lag +0 months (CONCURRENT     ): correlation = +0.031
  Lag +1 months (searches LAG   ): correlation = +0.010
  Lag +2 months (searches LAG   ): correlation = -0.041
  Lag +3 months (searches LAG   ): correlation = -0.044

  → Best correlation at lag +0: -999.

### South Korea

In [3]:
# KOREA ANALYSIS: ACLED EVENTS vs GOOGLE TRENDS

print("="*80)
print("SOUTH KOREA")
print("="*80)

# 1. FILTER ACLED DATA
# ---------------------

korea_acled = acled[
    (acled['COUNTRY'] == 'South Korea') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

korea_acled['WEEK'] = pd.to_datetime(korea_acled['WEEK'])

korea_acled['month'] = korea_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
korea_monthly = korea_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

print(f"✓ ACLED Data: {len(korea_monthly)} months")
print(f"  Date range: {korea_monthly['month'].min()} to {korea_monthly['month'].max()}")
print(f"  Total events: {korea_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {korea_monthly['FATALITIES'].sum():,}")


# 2. LOAD GOOGLE TRENDS FILES
# ----------------------------
korea_trends_files = {
    'South Korea protests': 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_South_Korea_protests.csv',
    'Korea demonstrations': 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_Korea_demonstrations.csv'
}

korea_trends_data = {}
for name, filepath in korea_trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        korea_trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"    ✗ Error loading {name}: {e}")
        

# 3. MERGE DATASETS
# -----------------
merged = korea_monthly.copy()
for name, df in korea_trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )
print(f"\n✓ Merged dataset: {len(merged)} months with {len(korea_trends_data)} search terms")


# 4. CORRELATION ANALYSIS
# -----------------------
print("\n" + "="*80)
print("CORRELATION ANALYSIS")
print("="*80)

correlations = []
for term in korea_trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

korea_corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + korea_corr_df.to_string(index=False))


# 5. TIME-LAG ANALYSIS
# --------------------
print("\n" + "="*80)
print("TIME-LAG ANALYSIS")
print("="*80)

korea_top_terms = korea_corr_df.head(3)['Search Term'].tolist()

for term in korea_top_terms:
    print(f"\n{term}:")
    valid_data = merged[['EVENTS', term]].dropna()
    best_corr = -999
    best_lag = 0
    
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

# 6. KEY PERIODS IDENTIFICATION
# -----------------------------
print("\n" + "="*80)
print("KEY PERIODS")
print("="*80)

print("\nTop 5 Event Spikes:")
korea_top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(korea_trends_data.keys())]
for idx, row in korea_top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in korea_trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

# 7. VISUALIZATION
# ---------------
print("\n" + "="*80)
print("CREATING VISUALIZATIONS")
print("="*80)

# Normalize data
korea_merged_normalized = merged.copy()
korea_merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
korea_merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
korea_plot_data = []
for _, row in korea_merged_normalized.iterrows():
    korea_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    korea_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    for term in korea_top_terms:
        if term in row and pd.notna(row[term]):
            korea_plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

korea_plot_df = pd.DataFrame(korea_plot_data)

# Main chart
chart = alt.Chart(korea_plot_data).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', title='Normalized Value (0-100)', scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', title='Metric', scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N', title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'South Korea: ACLED Events vs Google Search Interest (2020-2025)',
        'subtitle': 'Examining Country Events',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
).interactive()

chart.save('south_korea_acled_vs_trends.html')
print(f"✓ Saved: south_korea_acled_vs_trends.html")

# Display
chart


SOUTH KOREA
✓ ACLED Data: 94 months
  Date range: 2017-12-01 00:00:00 to 2025-09-01 00:00:00
  Total events: 14,176
  Total fatalities: 0
  ✓ Loaded: South Korea protests      - 70 months, max=100
  ✓ Loaded: Korea demonstrations      - 70 months, max=100

✓ Merged dataset: 94 months with 2 search terms

CORRELATION ANALYSIS

         Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
Korea demonstrations        0.016019                 NaN           69
South Korea protests       -0.031195                 NaN           69

TIME-LAG ANALYSIS

Korea demonstrations:
  Lag -3 months (searches LEAD  ): correlation = +0.107
  Lag -2 months (searches LEAD  ): correlation = +0.076
  Lag -1 months (searches LEAD  ): correlation = +0.131
  Lag +0 months (CONCURRENT     ): correlation = +0.016
  Lag +1 months (searches LAG   ): correlation = +0.131
  Lag +2 months (searches LAG   ): correlation = +0.076
  Lag +3 months (searches LAG   ): correlation = +0.107

  → Best correlation at lag

/Users/kayleeliu/Documents/Code/python/hashtags-for-change/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/kayleeliu/Documents/Code/python/hashtags-for-change/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/Users/kayleeliu/Documents/Code/python/hashtags-for-change/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/kayleeliu/Documents/Code/python/hashtags-for-change/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/Users/kayleeliu/Documents/Code/python/hashtags-for-change/.venv/lib/python3.12/site-packages/altair/vegalite/v5/api.py:295: UserWarning: data of type <class 'list'> no

SchemaValidationError: '[{'month': '2017-12-01T00:00:00', 'metric': 'ACLED Events', 'value': 7.584269662921349, 'type': 'Conflict Data', 'raw_value': 27}, {'month': '2017-12-01T00:00:00', 'metric': 'ACLED Fatalities', 'value': nan, 'type': 'Conflict Data', 'raw_value': 0}, {'month': '2018-01-01T00:00:00', 'metric': 'ACLED Events', 'value': 22.191011235955056, 'type': 'Conflict Data', 'raw_value': 79}, {'month': '2018-01-01T00:00:00', 'metric': 'ACLED Fatalities', 'value': nan, 'type': 'Conflict Data', 'raw_value': 0}, {'month': '2018-02-01T00:00:00', 'metric': 'ACLED Events', 'value': 15.44943820224719, 'type': 'Conflict Data', 'raw_value': 55}, {'month': '2018-02-01T00:00:00', 'metric': 'ACLED Fatalities', 'value': nan, 'type': 'Conflict Data', 'raw_value': 0}, {'month': '2018-03-01T00:00:00', 'metric': 'ACLED Events', 'value': 39.325842696629216, 'type': 'Conflict Data', 'raw_value': 140}, {'month': '2018-03-01T00:00:00', 'metric': 'ACLED Fatalities', 'value': nan, 'type': 'Conflict 

In [23]:
# 8. INDIVIDUAL COMPARISON CHARTS
# --------------------------------
for term in korea_top_terms:
    korea_term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    base = alt.Chart(korea_term_data).encode(
        x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', title='ACLED Events', axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    trends_line = base.mark_line(color='red', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='red'), scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'South Korea: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"south_korea_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

print("\n✓ South Korea analysis complete!")

✓ Saved: south_korea_korea_demonstrations_comparison.html
✓ Saved: south_korea_south_korea_protests_comparison.html

✓ South Korea analysis complete!


### Turkey

In [25]:
# TURKEY ANALYSIS: ACLED EVENTS vs GOOGLE TRENDS

print("="*80)
print("TURKEY ANALYSIS")
print("="*80)

# 1. FILTER ACLED DATA
# ---------------------

turkey_acled = acled[
    (acled['COUNTRY'] == 'Turkey') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

turkey_acled['WEEK'] = pd.to_datetime(turkey_acled['WEEK'])

turkey_acled['month'] = turkey_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
turkey_monthly = turkey_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()


print(f"✓ ACLED Data: {len(turkey_monthly)} months")
print(f"  Date range: {turkey_monthly['month'].min()} to {turkey_monthly['month'].max()}")
print(f"  Total events: {turkey_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {turkey_monthly['FATALITIES'].sum():,}")


# 2. LOAD GOOGLE TRENDS FILES
# ----------------------------
turkey_trends_files = {
    'Turkey conflict': 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_Turkey_conflict.csv',
    'Turkey violence': 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_Turkey_violence.csv',
    'Turkey military': 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_Turkey_military.csv'
}

turkey_trends_data = {}
for name, filepath in turkey_trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        turkey_trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"    ✗ Error loading {name}: {e}")
        

# 3. MERGE DATASETS
# -----------------
merged = turkey_monthly.copy()
for name, df in turkey_trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )
print(f"\n✓ Merged dataset: {len(merged)} months with {len(turkey_trends_data)} search terms")


# 4. CORRELATION ANALYSIS
# -----------------------
print("\n" + "="*80)
print("CORRELATION ANALYSIS")
print("="*80)

correlations = []
for term in turkey_trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

turkey_corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + turkey_corr_df.to_string(index=False))


# 5. TIME-LAG ANALYSIS
# --------------------
print("\n" + "="*80)
print("TIME-LAG ANALYSIS")
print("="*80)

turkey_top_terms = turkey_corr_df.head(3)['Search Term'].tolist()

for term in turkey_top_terms:
    print(f"\n{term}:")
    valid_data = merged[['EVENTS', term]].dropna()
    best_corr = -999
    best_lag = 0
    
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

# 6. KEY PERIODS IDENTIFICATION
# -----------------------------
print("\n" + "="*80)
print("KEY PERIODS")
print("="*80)

print("\nTop 5 Event Spikes:")
turkey_top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(turkey_trends_data.keys())]
for idx, row in turkey_top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in turkey_trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

# 7. VISUALIZATION
# ---------------
print("\n" + "="*80)
print("CREATING VISUALIZATIONS")
print("="*80)

# Normalize data
turkey_merged_normalized = merged.copy()
turkey_merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
turkey_merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
turkey_plot_data = []
for _, row in turkey_merged_normalized.iterrows():
    turkey_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    turkey_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    for term in turkey_top_terms:
        if term in row and pd.notna(row[term]):
            turkey_plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

turkey_plot_df = pd.DataFrame(turkey_plot_data)

# Main chart
chart = alt.Chart(turkey_plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', title='Normalized Value (0-100)', scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', title='Metric', scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N', title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'Turkey: ACLED Events vs Google Search Interest (2020-2025)',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
).interactive()

chart.save('turkey_acled_vs_trends.html')
print(f"✓ Saved: turkey_acled_vs_trends.html")

# Display
chart

# 8. INDIVIDUAL COMPARISON CHARTS
# --------------------------------
for term in turkey_top_terms:
    turkey_term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    base = alt.Chart(turkey_term_data).encode(
        x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', title='ACLED Events', axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    trends_line = base.mark_line(color='red', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='red'), scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'Turkey: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"turkey_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

print("\n✓ Turkey analysis complete!")

TURKEY ANALYSIS
✓ ACLED Data: 118 months
  Date range: 2015-12-01 00:00:00 to 2025-09-01 00:00:00
  Total events: 16,115
  Total fatalities: 3,109
  ✓ Loaded: Turkey conflict           - 70 months, max=100
  ✓ Loaded: Turkey violence           - 70 months, max=100
  ✓ Loaded: Turkey military           - 70 months, max=100

✓ Merged dataset: 118 months with 3 search terms

CORRELATION ANALYSIS

    Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
Turkey conflict        0.139099           -0.148544           69
Turkey military        0.096434           -0.200423           69
Turkey violence        0.048860           -0.163147           69

TIME-LAG ANALYSIS

Turkey conflict:
  Lag -3 months (searches LEAD  ): correlation = +0.275
  Lag -2 months (searches LEAD  ): correlation = +0.204
  Lag -1 months (searches LEAD  ): correlation = +0.191
  Lag +0 months (CONCURRENT     ): correlation = +0.139
  Lag +1 months (searches LAG   ): correlation = +0.191
  Lag +2 months (searches 

### Colombia

In [26]:
# COLOMBIA ANALYSIS: ACLED EVENTS vs GOOGLE TRENDS

print("="*80)
print("COLOMBIA ANALYSIS")
print("="*80)

# 1. FILTER ACLED DATA
# ---------------------

colombia_acled = acled[
    (acled['COUNTRY'] == 'Colombia') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

colombia_acled['WEEK'] = pd.to_datetime(colombia_acled['WEEK'])

colombia_acled['month'] = colombia_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
colombia_monthly = colombia_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()


print(f"✓ ACLED Data: {len(colombia_monthly)} months")
print(f"  Date range: {colombia_monthly['month'].min()} to {colombia_monthly['month'].max()}")
print(f"  Total events: {colombia_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {colombia_monthly['FATALITIES'].sum():,}")


# 2. LOAD GOOGLE TRENDS FILES
# ----------------------------
colombia_trends_files = {
    'Colombia violence': 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_Colombia_violence.csv',
    'Colombia conflict': 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_Colombia_conflict.csv'
}

colombia_trends_data = {}
for name, filepath in colombia_trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        colombia_trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"    ✗ Error loading {name}: {e}")
        

# 3. MERGE DATASETS
# -----------------
merged = colombia_monthly.copy()
for name, df in colombia_trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )
print(f"\n✓ Merged dataset: {len(merged)} months with {len(colombia_trends_data)} search terms")


# 4. CORRELATION ANALYSIS
# -----------------------
print("\n" + "="*80)
print("CORRELATION ANALYSIS")
print("="*80)

correlations = []
for term in colombia_trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

colombia_corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + colombia_corr_df.to_string(index=False))


# 5. TIME-LAG ANALYSIS
# --------------------
print("\n" + "="*80)
print("TIME-LAG ANALYSIS")
print("="*80)

colombia_top_terms = colombia_corr_df.head(3)['Search Term'].tolist()

for term in colombia_top_terms:
    print(f"\n{term}:")
    valid_data = merged[['EVENTS', term]].dropna()
    best_corr = -999
    best_lag = 0
    
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

# 6. KEY PERIODS IDENTIFICATION
# -----------------------------
print("\n" + "="*80)
print("KEY PERIODS")
print("="*80)

print("\nTop 5 Event Spikes:")
colombia_top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(colombia_trends_data.keys())]
for idx, row in colombia_top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in colombia_trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

# 7. VISUALIZATION
# ---------------
print("\n" + "="*80)
print("CREATING VISUALIZATIONS")
print("="*80)

# Normalize data
colombia_merged_normalized = merged.copy()
colombia_merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
colombia_merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
colombia_plot_data = []
for _, row in colombia_merged_normalized.iterrows():
    colombia_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    colombia_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    for term in colombia_top_terms:
        if term in row and pd.notna(row[term]):
            colombia_plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

colombia_plot_df = pd.DataFrame(colombia_plot_data)

# Main chart
chart = alt.Chart(colombia_plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', title='Normalized Value (0-100)', scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', title='Metric', scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N', title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'Colombia: ACLED Events vs Google Search Interest (2020-2025)',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
).interactive()

chart.save('colombia_acled_vs_trends.html')
print(f"✓ Saved: colombia_acled_vs_trends.html")

# Display
chart

# 8. INDIVIDUAL COMPARISON CHARTS
# --------------------------------
for term in colombia_top_terms:
    colombia_term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    base = alt.Chart(colombia_term_data).encode(
        x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', title='ACLED Events', axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    trends_line = base.mark_line(color='red', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='red'), scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'Colombia: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"colombia_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

print("\n✓ Colombia analysis complete!")

COLOMBIA ANALYSIS
✓ ACLED Data: 94 months
  Date range: 2017-12-01 00:00:00 to 2025-09-01 00:00:00
  Total events: 11,588
  Total fatalities: 4,836
  ✓ Loaded: Colombia violence         - 70 months, max=100
  ✓ Loaded: Colombia conflict         - 70 months, max=100

✓ Merged dataset: 94 months with 2 search terms

CORRELATION ANALYSIS

      Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
Colombia conflict        0.056278            0.207031           69
Colombia violence        0.045862            0.153865           69

TIME-LAG ANALYSIS

Colombia conflict:
  Lag -3 months (searches LEAD  ): correlation = +0.171
  Lag -2 months (searches LEAD  ): correlation = +0.038
  Lag -1 months (searches LEAD  ): correlation = +0.121
  Lag +0 months (CONCURRENT     ): correlation = +0.056
  Lag +1 months (searches LAG   ): correlation = +0.121
  Lag +2 months (searches LAG   ): correlation = +0.038
  Lag +3 months (searches LAG   ): correlation = +0.171

  → Best correlation at lag +

### Lebanon

In [27]:
# LEBANON ANALYSIS: ACLED EVENTS vs GOOGLE TRENDS

print("="*80)
print("LEBANON ANALYSIS")
print("="*80)

# 1. FILTER ACLED DATA
# ---------------------

lebanon_acled = acled[
    (acled['COUNTRY'] == 'Lebanon') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

lebanon_acled['WEEK'] = pd.to_datetime(lebanon_acled['WEEK'])

lebanon_acled['month'] = lebanon_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
lebanon_monthly = lebanon_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

print(f"✓ ACLED Data: {len(lebanon_monthly)} months")
print(f"  Date range: {lebanon_monthly['month'].min()} to {lebanon_monthly['month'].max()}")
print(f"  Total events: {lebanon_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {lebanon_monthly['FATALITIES'].sum():,}")


# 2. LOAD GOOGLE TRENDS FILES
# ----------------------------
lebanon_trends_files = {
    'Lebanon conflict': 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_Lebanon_conflict.csv',
    'Lebanon protests': 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_Lebanon_protests.csv',
    'Lebanon crisis': 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_Lebanon_crisis.csv'
}

lebanon_trends_data = {}
for name, filepath in lebanon_trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        lebanon_trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"    ✗ Error loading {name}: {e}")
        

# 3. MERGE DATASETS
# -----------------
merged = lebanon_monthly.copy()
for name, df in lebanon_trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )
print(f"\n✓ Merged dataset: {len(merged)} months with {len(lebanon_trends_data)} search terms")


# 4. CORRELATION ANALYSIS
# -----------------------
print("\n" + "="*80)
print("CORRELATION ANALYSIS")
print("="*80)

correlations = []
for term in lebanon_trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

lebanon_corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + lebanon_corr_df.to_string(index=False))


# 5. TIME-LAG ANALYSIS
# --------------------
print("\n" + "="*80)
print("TIME-LAG ANALYSIS")
print("="*80)

lebanon_top_terms = lebanon_corr_df.head(3)['Search Term'].tolist()

for term in lebanon_top_terms:
    print(f"\n{term}:")
    valid_data = merged[['EVENTS', term]].dropna()
    best_corr = -999
    best_lag = 0
    
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

# 6. KEY PERIODS IDENTIFICATION
# -----------------------------
print("\n" + "="*80)
print("KEY PERIODS")
print("="*80)

print("\nTop 5 Event Spikes:")
lebanon_top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(lebanon_trends_data.keys())]
for idx, row in lebanon_top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in lebanon_trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

# 7. VISUALIZATION
# ---------------
print("\n" + "="*80)
print("CREATING VISUALIZATIONS")
print("="*80)

# Normalize data
lebanon_merged_normalized = merged.copy()
lebanon_merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
lebanon_merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
lebanon_plot_data = []
for _, row in lebanon_merged_normalized.iterrows():
    lebanon_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    lebanon_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    for term in lebanon_top_terms:
        if term in row and pd.notna(row[term]):
            lebanon_plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

lebanon_plot_df = pd.DataFrame(lebanon_plot_data)

# Main chart
chart = alt.Chart(lebanon_plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', title='Normalized Value (0-100)', scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', title='Metric', scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N', title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'Lebanon: ACLED Events vs Google Search Interest (2020-2025)',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
).interactive()

chart.save('lebanon_acled_vs_trends.html')
print(f"✓ Saved: lebanon_acled_vs_trends.html")

# Display
chart

# 8. INDIVIDUAL COMPARISON CHARTS
# --------------------------------
for term in lebanon_top_terms:
    lebanon_term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    base = alt.Chart(lebanon_term_data).encode(
        x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', title='ACLED Events', axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    trends_line = base.mark_line(color='red', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='red'), scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'Lebanon: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"lebanon_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

print("\n✓ Lebanon analysis complete!")

LEBANON ANALYSIS
✓ ACLED Data: 118 months
  Date range: 2015-12-01 00:00:00 to 2025-09-01 00:00:00
  Total events: 11,869
  Total fatalities: 2,765
  ✓ Loaded: Lebanon conflict          - 70 months, max=100
  ✓ Loaded: Lebanon protests          - 70 months, max=100
  ✓ Loaded: Lebanon crisis            - 70 months, max=100

✓ Merged dataset: 118 months with 3 search terms

CORRELATION ANALYSIS

     Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
Lebanon conflict        0.570718            0.508761           69
Lebanon protests        0.023936            0.014331           69
  Lebanon crisis       -0.074656           -0.009259           69

TIME-LAG ANALYSIS

Lebanon conflict:
  Lag -3 months (searches LEAD  ): correlation = +0.702
  Lag -2 months (searches LEAD  ): correlation = +0.697
  Lag -1 months (searches LEAD  ): correlation = +0.634
  Lag +0 months (CONCURRENT     ): correlation = +0.571
  Lag +1 months (searches LAG   ): correlation = +0.634
  Lag +2 months (sea

### Nigeria

In [28]:
# NIGERIA ANALYSIS: ACLED EVENTS vs GOOGLE TRENDS

print("="*80)
print("NIGERIA ANALYSIS")
print("="*80)

# 1. FILTER ACLED DATA
# ---------------------

nigeria_acled = acled[
    (acled['COUNTRY'] == 'Nigeria') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

nigeria_acled['WEEK'] = pd.to_datetime(nigeria_acled['WEEK'])

nigeria_acled['month'] = nigeria_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
nigeria_monthly = nigeria_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()


print(f"✓ ACLED Data: {len(nigeria_monthly)} months")
print(f"  Date range: {nigeria_monthly['month'].min()} to {nigeria_monthly['month'].max()}")
print(f"  Total events: {nigeria_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {nigeria_monthly['FATALITIES'].sum():,}")


# 2. LOAD GOOGLE TRENDS FILES
# ----------------------------
nigeria_trends_files = {
    'Nigeria insurgency': 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_Nigeria_insurgency.csv',
    'Nigeria violence': 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_Nigeria_violence.csv',
    'Bokok haram': 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_Boko_Haram.csv'
}

nigeria_trends_data = {}
for name, filepath in nigeria_trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        nigeria_trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"    ✗ Error loading {name}: {e}")


# 3. MERGE DATASETS
# -----------------
merged = nigeria_monthly.copy()
for name, df in nigeria_trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )
print(f"\n✓ Merged dataset: {len(merged)} months with {len(nigeria_trends_data)} search terms")


# 4. CORRELATION ANALYSIS
# -----------------------
print("\n" + "="*80)
print("CORRELATION ANALYSIS")
print("="*80)

correlations = []
for term in nigeria_trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

nigeria_corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + nigeria_corr_df.to_string(index=False))


# 5. TIME-LAG ANALYSIS
# --------------------
print("\n" + "="*80)
print("TIME-LAG ANALYSIS")
print("="*80)

nigeria_top_terms = nigeria_corr_df.head(3)['Search Term'].tolist()

for term in nigeria_top_terms:
    print(f"\n{term}:")
    valid_data = merged[['EVENTS', term]].dropna()
    best_corr = -999
    best_lag = 0
    
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

# 6. KEY PERIODS IDENTIFICATION
# -----------------------------
print("\n" + "="*80)
print("KEY PERIODS")
print("="*80)

print("\nTop 5 Event Spikes:")
nigeria_top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(nigeria_trends_data.keys())]
for idx, row in nigeria_top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in nigeria_trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

# 7. VISUALIZATION
# ---------------
print("\n" + "="*80)
print("CREATING VISUALIZATIONS")
print("="*80)

# Normalize data
nigeria_merged_normalized = merged.copy()
nigeria_merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
nigeria_merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
nigeria_plot_data = []
for _, row in nigeria_merged_normalized.iterrows():
    nigeria_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    nigeria_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    for term in nigeria_top_terms:
        if term in row and pd.notna(row[term]):
            nigeria_plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

nigeria_plot_df = pd.DataFrame(nigeria_plot_data)

# Main chart
chart = alt.Chart(nigeria_plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', title='Normalized Value (0-100)', scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', title='Metric', scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N', title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'Nigeria: ACLED Events vs Google Search Interest (2020-2025)',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
).interactive()

chart.save('nigeria_acled_vs_trends.html')
print(f"✓ Saved: nigeria_acled_vs_trends.html")

# Display
chart

# 8. INDIVIDUAL COMPARISON CHARTS
# --------------------------------
for term in nigeria_top_terms:
    nigeria_term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    base = alt.Chart(nigeria_term_data).encode(
        x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', title='ACLED Events', axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    trends_line = base.mark_line(color='red', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='red'), scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'Nigeria: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"nigeria_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

print("\n✓ Nigeria analysis complete!")

NIGERIA ANALYSIS
✓ ACLED Data: 338 months
  Date range: 1997-01-01 00:00:00 to 2025-09-01 00:00:00
  Total events: 14,535
  Total fatalities: 42,297
  ✓ Loaded: Nigeria insurgency        - 70 months, max=100
  ✓ Loaded: Nigeria violence          - 70 months, max=100
  ✓ Loaded: Bokok haram               - 70 months, max=100

✓ Merged dataset: 338 months with 3 search terms

CORRELATION ANALYSIS

       Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
  Nigeria violence        0.097981           -0.031077           69
Nigeria insurgency       -0.038234            0.087509           69
       Bokok haram       -0.282388           -0.055765           69

TIME-LAG ANALYSIS

Nigeria violence:
  Lag -3 months (searches LEAD  ): correlation = +0.039
  Lag -2 months (searches LEAD  ): correlation = +0.018
  Lag -1 months (searches LEAD  ): correlation = +0.191
  Lag +0 months (CONCURRENT     ): correlation = +0.098
  Lag +1 months (searches LAG   ): correlation = +0.191
  Lag +2 mo

### Italy

In [29]:
# ITALY ANALYSIS: ACLED EVENTS vs GOOGLE TRENDS

print("="*80)
print("ITALY ANALYSIS")
print("="*80)

# 1. FILTER ACLED DATA
# ---------------------

italy_acled = acled[
    (acled['COUNTRY'] == 'Italy') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

italy_acled['WEEK'] = pd.to_datetime(italy_acled['WEEK'])

italy_acled['month'] = italy_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
italy_monthly = italy_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()


print(f"✓ ACLED Data: {len(italy_monthly)} months")
print(f"  Date range: {italy_monthly['month'].min()} to {italy_monthly['month'].max()}")
print(f"  Total events: {italy_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {italy_monthly['FATALITIES'].sum():,}")


# 2. LOAD GOOGLE TRENDS FILES
# ----------------------------
italy_trends_files = {
    'Italy protests': 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_Italy_protests.csv',
    'Italian demonstrations': 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_Italian_demonstrations.csv'
}

italy_trends_data = {}
for name, filepath in italy_trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        italy_trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"    ✗ Error loading {name}: {e}")
        

# 3. MERGE DATASETS
# -----------------
merged = italy_monthly.copy()
for name, df in italy_trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )
print(f"\n✓ Merged dataset: {len(merged)} months with {len(italy_trends_data)} search terms")


# 4. CORRELATION ANALYSIS
# -----------------------
print("\n" + "="*80)
print("CORRELATION ANALYSIS")
print("="*80)

correlations = []
for term in italy_trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

italy_corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + italy_corr_df.to_string(index=False))


# 5. TIME-LAG ANALYSIS
# --------------------
print("\n" + "="*80)
print("TIME-LAG ANALYSIS")
print("="*80)

italy_top_terms = italy_corr_df.head(3)['Search Term'].tolist()

for term in italy_top_terms:
    print(f"\n{term}:")
    valid_data = merged[['EVENTS', term]].dropna()
    best_corr = -999
    best_lag = 0
    
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

# 6. KEY PERIODS IDENTIFICATION
# -----------------------------
print("\n" + "="*80)
print("KEY PERIODS")
print("="*80)

print("\nTop 5 Event Spikes:")
italy_top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(italy_trends_data.keys())]
for idx, row in italy_top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in italy_trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

# 7. VISUALIZATION
# ---------------
print("\n" + "="*80)
print("CREATING VISUALIZATIONS")
print("="*80)

# Normalize data
italy_merged_normalized = merged.copy()
italy_merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
italy_merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
italy_plot_data = []
for _, row in italy_merged_normalized.iterrows():
    italy_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    italy_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    for term in italy_top_terms:
        if term in row and pd.notna(row[term]):
            italy_plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

italy_plot_df = pd.DataFrame(italy_plot_data)

# Main chart
chart = alt.Chart(italy_plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', title='Normalized Value (0-100)', scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', title='Metric', scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N', title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'Italy: ACLED Events vs Google Search Interest (2020-2025)',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
).interactive()

chart.save('italy_acled_vs_trends.html')
print(f"✓ Saved: italy_acled_vs_trends.html")

# Display
chart

# 8. INDIVIDUAL COMPARISON CHARTS
# --------------------------------
for term in italy_top_terms:
    italy_term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    base = alt.Chart(italy_term_data).encode(
        x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', title='ACLED Events', axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    trends_line = base.mark_line(color='red', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='red'), scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'Italy: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"italy_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

print("\n✓ Italy analysis complete!")

ITALY ANALYSIS
✓ ACLED Data: 70 months
  Date range: 2019-12-01 00:00:00 to 2025-09-01 00:00:00
  Total events: 9,350
  Total fatalities: 0
  ✓ Loaded: Italy protests            - 70 months, max=100
  ✓ Loaded: Italian demonstrations    - 70 months, max=100

✓ Merged dataset: 70 months with 2 search terms

CORRELATION ANALYSIS

           Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
        Italy protests        0.211648                 NaN           69
Italian demonstrations        0.054887                 NaN           69

TIME-LAG ANALYSIS

Italy protests:
  Lag -3 months (searches LEAD  ): correlation = +0.362
  Lag -2 months (searches LEAD  ): correlation = +0.338
  Lag -1 months (searches LEAD  ): correlation = +0.185
  Lag +0 months (CONCURRENT     ): correlation = +0.212
  Lag +1 months (searches LAG   ): correlation = +0.185
  Lag +2 months (searches LAG   ): correlation = +0.338
  Lag +3 months (searches LAG   ): correlation = +0.362

  → Best correlation at l

/Users/kayleeliu/Documents/Code/python/hashtags-for-change/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/kayleeliu/Documents/Code/python/hashtags-for-change/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/Users/kayleeliu/Documents/Code/python/hashtags-for-change/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/kayleeliu/Documents/Code/python/hashtags-for-change/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


### Afghanistan

In [30]:
# AFGHANISTAN ANALYSIS: ACLED EVENTS vs GOOGLE TRENDS

print("="*80)
print("AFGHANISTAN ANALYSIS")
print("="*80)

# 1. FILTER ACLED DATA
# ---------------------

afghanistan_acled = acled[
    (acled['COUNTRY'] == 'Afghanistan') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

afghanistan_acled['WEEK'] = pd.to_datetime(afghanistan_acled['WEEK'])

afghanistan_acled['month'] = afghanistan_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
afghanistan_monthly = afghanistan_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

print(f"✓ ACLED Data: {len(afghanistan_monthly)} months")
print(f"  Date range: {afghanistan_monthly['month'].min()} to {afghanistan_monthly['month'].max()}")
print(f"  Total events: {afghanistan_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {afghanistan_monthly['FATALITIES'].sum():,}")


# 2. LOAD GOOGLE TRENDS FILES
# ----------------------------
afghanistan_trends_files = {
    'Afghanistan conflict': 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_Afghanistan_conflict.csv',
    'Afghanistan military': 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_Afghanistan_military.csv',
    'Taliban': 'data/google_trends_raw/TIER2_COUNTRIES/google_trends_Taliban.csv'
}

# taliban_df = pd.read_csv('data/google_trends_raw/TIER2_COUNTRIES/google_trends_Taliban.csv', skiprows=1)


afghanistan_trends_data = {}
for name, filepath in afghanistan_trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        afghanistan_trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"    ✗ Error loading {name}: {e}")
        

# 3. MERGE DATASETS
# -----------------
merged = afghanistan_monthly.copy()
for name, df in afghanistan_trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )
print(f"\n✓ Merged dataset: {len(merged)} months with {len(afghanistan_trends_data)} search terms")


# 4. CORRELATION ANALYSIS
# -----------------------
print("\n" + "="*80)
print("CORRELATION ANALYSIS")
print("="*80)

correlations = []
for term in afghanistan_trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

afghanistan_corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + afghanistan_corr_df.to_string(index=False))


# 5. TIME-LAG ANALYSIS
# --------------------
print("\n" + "="*80)
print("TIME-LAG ANALYSIS")
print("="*80)

afghanistan_top_terms = afghanistan_corr_df.head(3)['Search Term'].tolist()

for term in afghanistan_top_terms:
    print(f"\n{term}:")
    valid_data = merged[['EVENTS', term]].dropna()
    best_corr = -999
    best_lag = 0
    
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

# 6. KEY PERIODS IDENTIFICATION
# -----------------------------
print("\n" + "="*80)
print("KEY PERIODS")
print("="*80)

print("\nTop 5 Event Spikes:")
afghanistan_top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(afghanistan_trends_data.keys())]
for idx, row in afghanistan_top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in afghanistan_trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

# 7. VISUALIZATION
# ---------------
print("\n" + "="*80)
print("CREATING VISUALIZATIONS")
print("="*80)

# Normalize data
afghanistan_merged_normalized = merged.copy()
afghanistan_merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
afghanistan_merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
afghanistan_plot_data = []
for _, row in afghanistan_merged_normalized.iterrows():
    afghanistan_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    afghanistan_plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    for term in afghanistan_top_terms:
        if term in row and pd.notna(row[term]):
            afghanistan_plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

afghanistan_plot_df = pd.DataFrame(afghanistan_plot_data)

# Main chart
chart = alt.Chart(afghanistan_plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', title='Normalized Value (0-100)', scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', title='Metric', scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N', title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'Afghanistan: ACLED Events vs Google Search Interest (2020-2025)',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
).interactive()


# Event Annotations

annotation_data = pd.DataFrame({
    'date': [pd.Timestamp('2021-08')],
    'label': ['Taliban takeover'],
    'text_y': [110]
})


rules = alt.Chart(annotation_data).mark_rule(
    strokeWidth=2, strokeDash=[5, 5], color='black', opacity=1.0
).encode(
    x='date:T'
)

labels = alt.Chart(annotation_data).mark_text(
    align='center',
    baseline='top',
    dx=10,
    dy=20,
    color='black',
    fontSize=13
).encode(
    x='date:T',
    y='text_y:Q',
    text='label:N'
)

chart = chart + rules + labels


chart.save('afghanistan_acled_vs_trends.html')
print(f"✓ Saved: afghanistan_acled_vs_trends.html")

# Display
chart

# 8. INDIVIDUAL COMPARISON CHARTS
# --------------------------------
for term in afghanistan_top_terms:
    afghanistan_term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    base = alt.Chart(afghanistan_term_data).encode(
        x=alt.X('month:T', title='Month', axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', title='ACLED Events', axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    trends_line = base.mark_line(color='red', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='red'), scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'Afghanistan: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"afghanistan_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

print("\n✓ Afghanistan analysis complete!")

AFGHANISTAN ANALYSIS
✓ ACLED Data: 106 months
  Date range: 2016-12-01 00:00:00 to 2025-09-01 00:00:00
  Total events: 23,895
  Total fatalities: 70,176
  ✓ Loaded: Afghanistan conflict      - 70 months, max=100
  ✓ Loaded: Afghanistan military      - 70 months, max=100
  ✓ Loaded: Taliban                   - 70 months, max=100

✓ Merged dataset: 106 months with 3 search terms

CORRELATION ANALYSIS

         Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
Afghanistan military       -0.042273            0.008746           69
             Taliban       -0.045831            0.009692           69
Afghanistan conflict       -0.188648           -0.115049           69

TIME-LAG ANALYSIS

Afghanistan military:
  Lag -3 months (searches LEAD  ): correlation = -0.037
  Lag -2 months (searches LEAD  ): correlation = -0.037
  Lag -1 months (searches LEAD  ): correlation = -0.039
  Lag +0 months (CONCURRENT     ): correlation = -0.042
  Lag +1 months (searches LAG   ): correlation = -0